In [86]:
import pandas as pd
import numpy as np

Набор данных - http://archive.ics.uci.edu/ml/datasets/Chemical+Composition+of+Ceramic+Samples

In [87]:
df = pd.read_csv('data/Chemical Composion of Ceramic.csv')
df

,Ceramic Name,Part,Na2O,MgO,Al2O3,SiO2,K2O,CaO,TiO2,Fe2O3,MnO,CuO,ZnO,PbO2,Rb2O,SrO,Y2O3,ZrO2,P2O5
0,FLQ-1-b,Body,0.62,0.38,19.61,71.99,4.84,0.31,0.07,1.18,630,10,70,10,430,0,40,80,90
1,FLQ-2-b,Body,0.57,0.47,21.19,70.09,4.98,0.49,0.09,1.12,380,20,80,40,430,-10,40,100,110
2,FLQ-3-b,Body,0.49,0.19,18.60,74.70,3.47,0.43,0.06,1.07,420,20,50,50,380,40,40,80,200
3,FLQ-4-b,Body,0.89,0.30,18.01,74.19,4.01,0.27,0.09,1.23,460,20,70,60,380,10,40,70,210
4,FLQ-5-b,Body,0.03,0.36,18.41,73.99,4.33,0.65,0.05,1.19,380,40,90,40,360,10,30,80,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,DY-M-3-g,Glaze,0.34,0.55,12.37,70.70,5.33,8.06,0.06,1.61,1250,10,90,30,250,520,30,140,690
84,DY-QC-1-g,Glaze,0.72,0.34,12.20,72.19,6.19,6.06,0.04,1.27,1700,60,110,10,270,540,40,120,630
85,DY-QC-2-g,Glaze,0.23,0.24,12.99,71.81,5.25,7.15,0.05,1.29,750,40,100,0,240,470,40,120,480
86,DY-QC-3-g,Glaze,0.14,0.46,12.62,69.16,4.34,11.03,0.05,1.20,920,40,90,20,230,470,40,130,1100


Проанализировать предметную область и выбрать метрику оценки качества кластеризации.

- Silhouette Score (Коэффициент силуэта): Эта метрика измеряет, насколько похож объект на объекты внутри своего кластера по сравнению с объектами из других кластеров. Значения варьируются от -1 до 1, где 1 означает, что объект идеально соответствует своему кластеру.
- Davies-Bouldin Index: Эта метрика оценивает среднее отношение внутрикластерного расстояния к межкластерному. Более низкое значение указывает на лучшее разделение кластеров.
- Calinski-Harabasz Index: Эта метрика оценивает плотность и разделимость кластеров. Чем выше значение, тем лучше разделение.

Отобрать признаки

In [88]:
# Отбор признаков (используем все числовые столбцы)
X = df.iloc[:, 2:].values  # Преобразуем в NumPy массив для удобства дальнейших вычислений

# Нормализация данных
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_scaled = (X - X_min) / (X_max - X_min)
X_scaled

array([[0.31891892, 0.248     , 0.54743083, ..., 0.33333333, 0.08823529,
        0.02564103],
       [0.29189189, 0.32      , 0.65151515, ..., 0.33333333, 0.14705882,
        0.03846154],
       [0.24864865, 0.096     , 0.48089592, ..., 0.33333333, 0.08823529,
        0.09615385],
       ...,
       [0.10810811, 0.136     , 0.1113307 , ..., 0.33333333, 0.20588235,
        0.27564103],
       [0.05945946, 0.312     , 0.08695652, ..., 0.33333333, 0.23529412,
        0.67307692],
       [0.05945946, 0.448     , 0.19433465, ..., 0.33333333, 0.17647059,
        0.41025641]])

Реализовать алгоритм k-means++

In [89]:
def initialize_centroids(X, k):
    n_samples, n_features = X.shape
    centroids = np.zeros((k, n_features))

    # Первый центр выбирается случайно
    centroids[0] = X[np.random.choice(range(n_samples))]

    # Выбор следующих центров
    for i in range(1, k):
        distances = np.array([min(np.sum((x - centroids[:i])**2, axis=1)) for x in X])
        probabilities = distances / distances.sum()
        centroids[i] = X[np.random.choice(range(n_samples), p=probabilities)]

    return centroids

def assign_clusters(X, centroids):
    distances = np.array([[np.sum((x - centroid)**2) for centroid in centroids] for x in X])
    return np.argmin(distances, axis=1)

def update_centroids(X, labels, k):
    centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
    return centroids

def kmeans(X, k, max_iters=100):
    centroids = initialize_centroids(X, k)
    for _ in range(max_iters):
        labels = assign_clusters(X, centroids)
        new_centroids = update_centroids(X, labels, k)
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids
    return labels, centroids

# Пример использования
k = 3  # Предполагаемое количество кластеров
kmeans_labels, kmeans_centroids = kmeans(X_scaled, k)

Реализовать алгоритм DBSCAN

In [90]:
def region_query(X, point_idx, eps):
    neighbors = []
    for i in range(len(X)):
        if np.linalg.norm(X[i] - X[point_idx]) < eps:
            neighbors.append(i)
    return neighbors

def expand_cluster(X, labels, point_idx, cluster_id, eps, min_samples):
    neighbors = region_query(X, point_idx, eps)
    if len(neighbors) < min_samples:
        labels[point_idx] = -1  # Шум
        return False
    else:
        labels[point_idx] = cluster_id
        i = 0
        while i < len(neighbors):
            neighbor_idx = neighbors[i]
            if labels[neighbor_idx] == -1:
                labels[neighbor_idx] = cluster_id
            elif labels[neighbor_idx] == 0:
                labels[neighbor_idx] = cluster_id
                new_neighbors = region_query(X, neighbor_idx, eps)
                if len(new_neighbors) >= min_samples:
                    neighbors.extend(new_neighbors)
            i += 1
        return True

def dbscan(X, eps, min_samples):
    labels = np.zeros(len(X))  # Изначально все точки не присвоены ни к одному кластеру
    cluster_id = 0
    for i in range(len(X)):
        if labels[i] == 0:
            if expand_cluster(X, labels, i, cluster_id + 1, eps, min_samples):
                cluster_id += 1
    return labels

# Пример использования
eps = 0.5
min_samples = 5
dbscan_labels = dbscan(X_scaled, eps, min_samples)

Проверить качество кластеризации по выбранной метрике

In [91]:
def silhouette_score(X, labels):
    unique_labels = np.unique(labels)
    silhouette_scores = []
    for i in range(len(X)):
        same_cluster = X[labels == labels[i]]
        other_clusters = [X[labels == l] for l in unique_labels if l != labels[i]]
        
        a = np.mean([np.linalg.norm(X[i] - point) for point in same_cluster if not np.array_equal(X[i], point)])
        b = np.min([np.mean([np.linalg.norm(X[i] - point) for point in cluster]) for cluster in other_clusters])
        
        silhouette = (b - a) / max(a, b)
        silhouette_scores.append(silhouette)
        
    return np.mean(silhouette_scores)

# Оценка качества для K-means++
kmeans_silhouette = silhouette_score(X_scaled, kmeans_labels)
print(f'Silhouette Score для K-means++: {kmeans_silhouette}')

# Оценка качества для DBSCAN
dbscan_silhouette = silhouette_score(X_scaled[dbscan_labels > 0], dbscan_labels[dbscan_labels > 0])
print(f'Silhouette Score для DBSCAN: {dbscan_silhouette}')


Silhouette Score для K-means++: 0.27136540900604267
Silhouette Score для DBSCAN: 0.5995638280018996
